In [1]:
import numpy as np
import tensorflow as tf
import keras
import os
from tqdm import tqdm
import time

In [2]:
class DataGenerator(tf.compat.v2.keras.utils.Sequence):
    def __init__(self, data_path, ann_path, list_files, list_ann_files, 
                 batch_size=64, dim=(3000,1), n_classes=5, shuffle=True):
        # Constructor of the data generator.
        self.dim = dim
        self.batch_size = batch_size
        self.data_path = data_path
        self.ann_path = ann_path
        self.list_files = list_files
        self.list_ann_files = list_ann_files
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.get_cnts() #Get the data count for each file        
        self.on_epoch_end() #Initialize file indexes        
        

    def __len__(self):
        # Denotes the number of batches per epoch
        return int((self.total_len+1) / self.batch_size)
    

    def __getitem__(self, index):
        
        start = index*self.batch_size
        end = min((index+1)*self.batch_size, self.total_len)
        
        X = np.empty((end - start,) + self.dim, dtype=np.float32)
        y = np.empty((end - start,), dtype=np.int32)
        
        curr_file_idx, accum_start, accum_end = self.get_accum_idx(index)
        
        curr_file = self.list_files[self.file_indexes[curr_file_idx]]
        curr_ann_file = self.list_ann_files[self.file_indexes[curr_file_idx]]
        data_index = self.data_indexes[self.file_indexes[curr_file_idx]]
        
        curr_np = np.load(os.path.join(self.data_path, curr_file))
        curr_ann = np.load(os.path.join(self.ann_path, curr_ann_file))
        curr_np = curr_np[data_index]
        curr_ann = curr_ann[data_index]
        
        

        
        X_1 = curr_np[start - accum_start:end - accum_start] 
        y_1 = curr_ann[start - accum_start:end - accum_start]
        from_curr = min(accum_end - start, end - start)
        X[:from_curr] = np.expand_dims(X_1, axis=-1)
        y[:from_curr] = y_1
        
        if end > accum_end:
            curr_file_idx += 1
            accum_start = accum_end
            accum_end += self.list_cnt[self.file_indexes[curr_file_idx]]
            curr_file = self.list_files[self.file_indexes[curr_file_idx]]            
            data_index = self.data_indexes[self.file_indexes[curr_file_idx]]
            
            
            curr_ann_file = self.list_ann_files[self.file_indexes[curr_file_idx]]
            curr_np = np.load(os.path.join(self.data_path, curr_file))
            curr_ann = np.load(os.path.join(self.ann_path, curr_ann_file))

            curr_np = curr_np[data_index]
            curr_ann = curr_ann[data_index]
            #curr_np = curr_np.reshape(-1, 3000, 1)
            
            #curr_np = curr_np[1:-1]
            #curr_ann = curr_ann[1:-1]
            
            X_2 = curr_np[:end - accum_start]
            y_2 = curr_ann[:end - accum_start]
            X[from_curr:] = np.expand_dims(X_2, axis=-1)
            y[from_curr:] = y_2
        
        '''
        # Normalize data(MinMax)
        rng = np.max(X, axis=1) - np.min(X, axis=1) #X shape: (B, 3000, 1), rng: (B, 1)
        rng = np.expand_dims(rng, axis=1) #(B, 1, 1)
        X = (X - np.expand_dims(np.min(X, axis=1),axis=1)) / (rng + 1e-8)
        '''                
        return X, y
    
    def get_accum_idx(self, index):
        curr_file_idx = 0
        accum_start = 0
        accum_end = self.list_cnt[self.file_indexes[0]]
        for i in range(len(self.file_indexes)):
            if index * self.batch_size < accum_end:
                curr_file_idx = i                
                break            
            accum_start += self.list_cnt[self.file_indexes[i]]
            accum_end += self.list_cnt[self.file_indexes[i+1]]
        
        return curr_file_idx, accum_start, accum_end
        
    def on_epoch_end(self):        
        self.curr_file_idx = 0
        # This function is called at the end of each epoch.
        self.file_indexes = np.arange(len(self.list_files)) #This is necessary to shuffle files
        self.data_indexes = [np.arange(cnt) for cnt in self.list_cnt]
        if self.shuffle == True:
            np.random.shuffle(self.file_indexes)
            for i in range(len(self.list_cnt)):
                np.random.shuffle(self.data_indexes[i]) 
            
        #self.accum_start = 0 
        #self.accum_end = self.list_cnt[self.file_indexes[0]]                 
            
    def get_cnts(self):
        list_cnt = []
        for f in self.list_files:
            temp_np = np.load(os.path.join(self.data_path, f))
            cnt_data = temp_np.shape[0] 
            list_cnt.append(cnt_data)
            
        self.list_cnt = list_cnt
        self.total_len = sum(list_cnt)    

In [3]:
#curr_path = os.getcwd() + '/'
PROCESSED_DATA_PATH = os.path.join('/home','aiot','data','origin_npy')
save_signals_path = os.path.join(PROCESSED_DATA_PATH,'signals_SC_filtered')
save_annotations_path = os.path.join(PROCESSED_DATA_PATH,'annotations_SC')

In [4]:
def match_annotations_npy(dirname, filename):
    search_filename = filename.split('-')[0][:-2]
    file_list = os.listdir(dirname)
    filenames = [file for file in file_list if search_filename in file if file.endswith('.npy')]

    return filenames

In [5]:
dim_HT1D = (3000,1)
n_classes=6
epochs = 50
bs = 64
BASE_LEARNING_RATE = 1e-3
list_files = [f for f in os.listdir(save_signals_path) if f.endswith('.npy')]

In [6]:
def read_csv_to_list(filepath):
    import csv
    with open(filepath, newline='') as csvfile:
        spamreader = csv.reader(csvfile, delimiter=',')
        list_filepath = [row[0] for row in spamreader]
    return list_filepath

In [7]:
SC_train = os.path.join('/home','aiot','data','origin_npy','SC_train.csv')
SC_test = os.path.join('/home','aiot','data','origin_npy','SC_test.csv')

list_files_train = read_csv_to_list(SC_train)
list_files_test = read_csv_to_list(SC_test)

list_files_train = [f + '.npy' for f in list_files_train]
list_files_test = [f + '.npy' for f in list_files_test]

list_ann_files_train = []
list_ann_files_test = []
for f in list_files_train:
    ann_file = match_annotations_npy(save_annotations_path, f)
    list_ann_files_train.append(ann_file[0])
    
for f in list_files_test:
    ann_file = match_annotations_npy(save_annotations_path, f)
    list_ann_files_test.append(ann_file[0])



#split_cnt = int(len(list_files) * 0.8)
#list_files_train = list_files[:split_cnt]
#list_files_test = list_files[split_cnt:]
#list_ann_files_train = list_ann_files[:split_cnt]
#list_ann_files_test = list_ann_files[split_cnt:]

#list_files_train = list_files[:5]
#list_files_test = list_files[80:90]
#list_ann_files_train = list_ann_files[0:5]
#list_ann_files_test = list_ann_files[80:90]

In [8]:
train_generator = DataGenerator(save_signals_path, save_annotations_path, list_files_train, list_ann_files_train, 
                          batch_size=bs, dim=dim_HT1D, n_classes=n_classes, shuffle=True)

In [9]:
test_generator = DataGenerator(save_signals_path, save_annotations_path, list_files_test, list_ann_files_test, 
                          batch_size=bs, dim=dim_HT1D, n_classes=n_classes, shuffle=False)

In [ ]:
# Calculate class weight
# Tested loss with class weight, but doesn't improve the accuracy

from collections import defaultdict
cnt_class = defaultdict(int)
for x, y in train_generator:
    unique, counts = np.unique(y, return_counts=True)
    for i, cnt in zip(unique, counts):
        cnt_class[i] += cnt
cnt_class_np = np.array(list(cnt_class.values()))
class_weight = sum(cnt_class_np)/(n_classes * cnt_class_np)


In [11]:
from tensorflow.keras.layers import BatchNormalization, LSTM, Conv1D, ReLU, Input, Dense, Flatten, RepeatVector, Reshape, Dropout, add        

In [12]:
class MultiheadAttention(tf.keras.layers.Layer):
    def __init__(self, n_heads, embed_dim):
        super().__init__()
        self.n_heads = n_heads
        self.embed_dim = embed_dim
        self.emb = Dense(embed_dim * n_heads * 3, use_bias=True)         
        d = tf.cast(embed_dim, dtype=tf.float32)    
        self.scaling = 1/tf.math.sqrt(d)

    def call(self, inputs):    
        """
        inputs: (B, num_seed, features)
        """
        #num_seed = tf.shape(inputs)[1]
        embedding = self.emb(inputs) # (B, n, d * h * 3)            
        heads = Reshape((-1, self.embed_dim, self.n_heads, 3))(embedding) #(B, n, d, h, 3)
        

        heads = tf.transpose(heads, perm=[0,4,3,1,2]) # (B, 3, h, n, d)
        q = heads[:,0,:,:,:] #(B, h, n, d)
        k = heads[:,1,:,:,:] #(B, h, n, d)
        v = heads[:,2,:,:,:] #(B, h, n, d)
        
        qk = tf.matmul(q, k, transpose_b=True) # (B, h, n, n)    
        qk = tf.keras.backend.softmax(qk) * self.scaling # (B, h, n, n)            
        attn = qk / self.scaling
        
        output = tf.matmul(attn, v) # (B, h, n, d)
        output = tf.transpose(output, perm=[0,2,1,3]) #(B, n, h, d)
        output = Reshape((-1, self.embed_dim * self.n_heads))(output)
        return  output

In [13]:
class conv1d_block(tf.keras.layers.Layer):
    def __init__(self, filters=64, kernel_size=100, strides=1, padding='valid'):
        super().__init__()
        self.conv = Conv1D(filters=filters, kernel_size=kernel_size, padding=padding, strides=strides)
        self.bn = BatchNormalization(axis=-1)
        self.relu = ReLU()
    def call(self, x):
        x = self.relu(self.bn(self.conv(x)))
        return x

In [23]:
class LSTMAttention(tf.keras.Model):
    def __init__(self, input_shape=(3000,1), n_classes=6):
        super().__init__()
        #self.input = Input(shape=input_shape)
        
        self.conv0_1 = conv1d_block(filters=32, kernel_size=300, strides=10)
        self.conv0_2 = conv1d_block(filters=64, kernel_size=5, strides=3)
        #self.squeeze_conv1 = conv1d_block(filters=64, kernel_size=179, strides=1)
        #self.squeeze_conv2 = conv1d_block(filters=256, kernel_size=1, strides=1)
        self.lstm1 = LSTM(32)
        self.dense1 = Dense(256)
        self.mha1 = MultiheadAttention(n_heads=8, embed_dim=32)
        self.conv2_1 = conv1d_block(filters=256, kernel_size=1, strides=1, padding='same')
        self.conv2_2 = conv1d_block(filters=256, kernel_size=1, strides=1, padding='same')
        self.conv2_3 = conv1d_block(filters=256, kernel_size=1, strides=1, padding='same')
        self.dropout2_1 = Dropout(0.1)
        
        #self.mha2 = MultiheadAttention(n_heads=8, embed_dim=32)
        #self.conv2_4 = conv1d_block(filters=512, kernel_size=1, strides=1, padding='same')
        #self.conv2_5 = conv1d_block(filters=512, kernel_size=1, strides=1, padding='same')
        #self.conv2_6 = conv1d_block(filters=512, kernel_size=1, strides=1, padding='same')        
        
        self.conv1_1 = conv1d_block(filters=128, kernel_size=5, strides=2)
        self.conv1_2 = conv1d_block(filters=128, kernel_size=5, strides=1, padding='same')
        self.conv1_3 = conv1d_block(filters=128, kernel_size=5, strides=1, padding='same')
        self.dropout1_1 = Dropout(0.1)
        
        self.conv1_4 = conv1d_block(filters=256, kernel_size=5, strides=2)
        self.conv1_5 = conv1d_block(filters=256, kernel_size=5, strides=1, padding='same')
        self.conv1_6 = conv1d_block(filters=256, kernel_size=5, strides=1, padding='same')
        self.dropout1_2 = Dropout(0.1)
                
        self.dropout3 = Dropout(0.1)
        self.final_conv1 = conv1d_block(filters=256, kernel_size=5, strides=1, padding='same')
        self.final_conv2 = conv1d_block(filters=256, kernel_size=5, strides=1, padding='same')
        self.final_conv3 = conv1d_block(filters=256, kernel_size=5, strides=1, padding='same')
        
        self.fc = Dense(n_classes, activation='softmax')
        
    def call(self, x):
        #x = self.input(x)        
        x = self.conv0_1(x)
        
        y = self.lstm1(x)
        y = self.dense1(y)
        y = Reshape((1,1,256))(y)
        identity = y
        y = self.mha1(y)
        y = self.conv2_1(y)
        y = self.conv2_2(y)
        y = self.conv2_3(y)
        y = self.dropout2_1(y)
        y = add([identity, y])
        
        
        x = self.conv0_2(x)
        #y = self.squeeze_conv1(x)
        #y = self.squeeze_conv2(y)       
        
        x = self.conv1_1(x)
        identity = x
        x = self.conv1_2(x)
        x = self.conv1_3(x)
        x = self.dropout1_1(x)
        x = add([identity, x])
        
        x = self.conv1_4(x)
        identity = x
        x = self.conv1_5(x)
        x = self.conv1_6(x)
        x = self.dropout1_2(x)
        x = add([identity, x])
        
        
        y = Reshape((256,))(y)
        y = RepeatVector(20)(y)
        
        x = add([x, y])
        x = self.dropout3(x)
               
        identity = x
        x = self.final_conv1(x)        
        x = self.final_conv2(x)        
        x = self.final_conv3(x) 
        x = add([identity, x])
        x = Flatten()(x)
        x = self.fc(x)
        
        return x

In [24]:
model = LSTMAttention()

In [25]:
x = np.random.random((1,3000,1))
x = tf.convert_to_tensor(x)
model(x)

<tf.Tensor: shape=(1, 6), dtype=float32, numpy=
array([[0.16747847, 0.165163  , 0.17343563, 0.15941907, 0.16582629,
        0.16867754]], dtype=float32)>

In [26]:
model.summary()

Model: "lstm_attention_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_block_14 (conv1d_bloc multiple                  9760      
_________________________________________________________________
conv1d_block_15 (conv1d_bloc multiple                  10560     
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8320      
_________________________________________________________________
dense_3 (Dense)              multiple                  8448      
_________________________________________________________________
multihead_attention_1 (Multi multiple                  197376    
_________________________________________________________________
conv1d_block_16 (conv1d_bloc multiple                  66816     
_________________________________________________________________
conv1d_block_17 (conv1d_bloc multiple             

In [27]:
def get_current_lr(epoch):
    lr = BASE_LEARNING_RATE
    for _ in range(epoch // 10):
        lr *= 0.1
    return lr

def adjust_learning_rate(optimizer, epoch):
    lr = get_current_lr(epoch)
    optimizer.learning_rate = lr

In [ ]:
def weighted_categorical_crossentropy(weights):
    """
    A weighted version of keras.objectives.categorical_crossentropy
    
    Variables:
        weights: numpy array of shape (C,) where C is the number of classes
    
    Usage:
        weights = np.array([0.5,2,10]) # Class one at 0.5, class 2 twice the normal weights, class 3 10x.
        loss = weighted_categorical_crossentropy(weights)
        model.compile(loss=loss,optimizer='adam')
    """
    
    weights = K.variable(weights)
        
    def loss(y_true, y_pred):
        bs = y_pred.shape[0]
        # scale predictions so that the class probas of each sample sum to 1
        y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
        # clip to prevent NaN's and Inf's
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        # calc
        loss = y_true * K.log(y_pred) * weights
        #loss = -K.sum(loss, -1)
        loss = -K.sum(loss) / bs
        return loss
    
    return loss

In [28]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
#loss_fn = weighted_categorical_crossentropy(weights=class_weight)

In [29]:
ckpt = tf.train.Checkpoint(step=tf.Variable(1), optimizer=optimizer, net=model)
manager = tf.train.CheckpointManager(ckpt, './ckpt_Advanced_Conv1D', max_to_keep=1)

In [30]:
start_epoch = 0
#if manager.latest_checkpoint:
#    ckpt.restore(manager.latest_checkpoint)
#    start_epoch = ckpt.step.numpy()-1

In [31]:
best_test_acc = 0.0
for e in range(start_epoch, epochs):
    correct, total_cnt, total_loss = 0.0, 0.0, 0.0
    print('-'*20, 'Epoch ' + str(e) + '-'*20)
    adjust_learning_rate(optimizer, e)
    start = time.time()
    for idx, (x, y) in enumerate(train_generator): 
        #y_onehot = tf.one_hot(y, depth=n_classes)
        with tf.GradientTape() as tape:
            y_pred = model(x, training=True)
            #loss = loss_fn(y_onehot, y_pred)
            loss = loss_fn(y, y_pred)
        
        total_cnt += y_pred.shape[0]
        y_pred_cls = tf.math.argmax(y_pred, axis=-1)
        correct += tf.reduce_sum(tf.cast(tf.equal(y_pred_cls, y), tf.float32))
        total_loss += loss * y_pred.shape[0]
        if (idx + 1) % 10 == 0:
            print("[%d / %d] Training loss: %.6f, Training acc: %.3f"%
                  (idx+1, len(train_generator), total_loss / total_cnt, correct / total_cnt),end='\r', flush=True)
        grads = tape.gradient(loss, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
    print("")
    print("Training time: %.2f sec "%(time.time() - start))
    
    start = time.time()
    
    correct, total_cnt, total_loss = 0.0, 0.0, 0.0
    for idx, (x, y) in enumerate(test_generator):
        y_pred = model(x, training=False)
        y_pred_cls = tf.math.argmax(y_pred, axis=-1)
        correct += tf.reduce_sum(tf.cast(tf.equal(y_pred_cls, y), tf.float32))
        total_cnt += y_pred.shape[0]
        y = tf.cast(y, dtype=tf.int32)
        y_onehot = tf.one_hot(y, depth=n_classes)
        #total_loss += loss_fn(y_onehot, y_pred).numpy() * y_pred.shape[0]
        total_loss += loss_fn(y, y_pred).numpy() * y_pred.shape[0]
            
        test_acc = correct / total_cnt
        test_loss = total_loss / total_cnt
        if (idx + 1) % 10 == 0:
            print("[%d / %d] test loss: %.6f, test accuracy: %.3f"%
                  (idx+1, len(test_generator), test_loss, test_acc),end='\r', flush=True)
    print("")
    print("Eval time: %.2f sec"%(time.time() - start))
    ckpt.step.assign_add(1)
    if test_acc > best_test_acc:
        save_path = manager.save()
        print("Saved checkpoint for step {}: {}".format(int(ckpt.step), save_path))
    

-------------------- Epoch 0--------------------
[1160 / 1162] Training loss: 0.610771, Training acc: 0.823
Training time: 606.03 sec 
[510 / 515] test loss: 1.595881, test accuracy: 0.755
Eval time: 99.09 sec
Saved checkpoint for step 2: ./ckpt_Advanced_Conv1D/ckpt-1
-------------------- Epoch 1--------------------
[1160 / 1162] Training loss: 0.305751, Training acc: 0.900
Training time: 605.64 sec 
[510 / 515] test loss: 1.143505, test accuracy: 0.794
Eval time: 99.35 sec
Saved checkpoint for step 3: ./ckpt_Advanced_Conv1D/ckpt-2
-------------------- Epoch 2--------------------
[1160 / 1162] Training loss: 0.236334, Training acc: 0.919
Training time: 606.34 sec 
[510 / 515] test loss: 1.247552, test accuracy: 0.813
Eval time: 99.47 sec
Saved checkpoint for step 4: ./ckpt_Advanced_Conv1D/ckpt-3
-------------------- Epoch 3--------------------
[1160 / 1162] Training loss: 0.217462, Training acc: 0.924
Training time: 605.93 sec 
[510 / 515] test loss: 1.528251, test accuracy: 0.797
Eval

KeyboardInterrupt: 